In [1]:
from aco import ACO
import numpy as np
import logging
from gen_inst import OPInstance, load_dataset
import torch

N_ANTS = 30
N_ITERATIONS = [1, 10, 30, 50, 80, 100]

def heuristics_reevo(prize: np.ndarray, distance: np.ndarray, maxlen: float) -> np.ndarray:

    n = prize.shape[0]
    heuristics = np.zeros((n, n))

    # Calculate the prize-to-distance ratio with a power transformation
    prize_distance_ratio = np.power(prize / distance, 3)

    # Find the indices of valid edges based on the distance constraint
    valid_edges = np.where(distance <= maxlen)

    # Assign the prize-to-distance ratio to the valid edges
    heuristics[valid_edges] = prize_distance_ratio[valid_edges]

    return heuristics

def solve(inst: OPInstance):
    heu = heuristics_reevo(np.array(inst.prize), np.array(inst.distance), inst.maxlen) + 1e-9
    assert tuple(heu.shape) == (inst.n, inst.n)
    heu[heu < 1e-9] = 1e-9
    heu = torch.from_numpy(heu)
    aco = ACO(inst.prize, inst.distance, inst.maxlen, heu, N_ANTS)

    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj, _ = aco.run(N_ITERATIONS[i])
        else:
            obj, _ = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        results.append(obj.item())
    return results


In [2]:
for problem_size in [50]:
    dataset_path = f"./dataset/val{problem_size}_dataset.npz"
    dataset = load_dataset(dataset_path)
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path}")

    objs = []
    for i, instance in enumerate(dataset):
        obj = solve(instance)
        objs.append(obj)
    
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Average for 50, 1 iterations: 12.462335858585858
[*] Average for 50, 10 iterations: 14.484812184343431
[*] Average for 50, 30 iterations: 15.15307922979798
[*] Average for 50, 50 iterations: 15.31330018939394
[*] Average for 50, 80 iterations: 15.379081439393937
[*] Average for 50, 100 iterations: 15.39080018939394

